# Sistem Rekomendasi Film Menggunakan Collaborative Filtering

## Project Overview
Sistem rekomendasi film bertujuan membantu pengguna menemukan film yang sesuai dengan preferensi mereka secara otomatis. Dengan banyaknya pilihan film, sistem ini dapat meningkatkan pengalaman pengguna dan membantu platform streaming meningkatkan engagement.

## Business Understanding
Permasalahan: Pengguna sering kesulitan memilih film yang sesuai dengan selera mereka di antara ribuan pilihan.
Solusi: Membangun sistem rekomendasi yang dapat memberikan saran film secara personal menggunakan pendekatan 

- **Collaborative Filtering**: Merekomendasikan film berdasarkan pola rating pengguna lain yang mirip.

### Contoh penggunaan Collaborative Filtering (User-Based) dalam dataset ini

Pada pendekatan ini, sistem merekomendasikan film berdasarkan pola rating pengguna lain yang mirip. Model yang digunakan adalah user-based collaborative filtering dengan cosine similarity. Data rating dibagi menjadi data train dan validasi untuk menghindari data leakage dan memungkinkan evaluasi model.

In [ ]:
# Collaborative Filtering: User-Based

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.model_selection import train_test_split

# Membagi data ratings menjadi 80% train dan 20% validasi
ratings_train, ratings_val = train_test_split(
    ratings, test_size=0.2, random_state=42, shuffle=True
)

print("Jumlah data train:", ratings_train.shape[0])
print("Jumlah data validasi:", ratings_val.shape[0])

# Gunakan ratings_train untuk membangun user-item matrix
user_item_matrix = ratings_train.pivot_table(index='UserID', columns='MovieID', values='Rating')
user_item_matrix_filled = user_item_matrix.fillna(0)


# Hitung similarity antar user
user_sim = cosine_similarity(user_item_matrix_filled)
user_sim_df = pd.DataFrame(user_sim, index=user_item_matrix.index, columns=user_item_matrix.index)

def recommend_collaborative(user_id, top_n=5):
    if user_id not in user_item_matrix.index:
        print("User tidak ditemukan.")
        return
    # Cari user paling mirip (selain dirinya sendiri)
    sim_scores = user_sim_df[user_id].drop(user_id).sort_values(ascending=False)
    top_user = sim_scores.index[0]
    # Cari film yang sudah dirating user mirip, tapi belum dirating user target
    user_movies = set(user_item_matrix.loc[user_id].dropna().index)
    similar_user_movies = set(user_item_matrix.loc[top_user].dropna().index)
    recommend_movies = list(similar_user_movies - user_movies)
    # Ambil top_n film dengan rating tertinggi dari user mirip
    top_movies = user_item_matrix.loc[top_user, recommend_movies].sort_values(ascending=False).head(top_n)
    return movies[movies['MovieID'].isin(top_movies.index)][['Title', 'Genres']]

# Contoh rekomendasi untuk user dengan UserID=1
print("Rekomendasi film untuk UserID=1 (Collaborative Filtering):")
display(recommend_collaborative(2))

### Content-Based Filtering dengan TF-IDF

Pada pendekatan ini, sistem merekomendasikan film berdasarkan kemiripan konten, yaitu genre film. Genre diubah menjadi representasi numerik menggunakan TF-IDF (Term Frequency-Inverse Document Frequency), lalu dihitung kemiripan antar film menggunakan cosine similarity. Rekomendasi diberikan berdasarkan film yang paling mirip dengan film yang dipilih pengguna.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Membuat fitur genre sebagai string (tanpa tanda '|')
movies['Genres_str'] = movies['Genres'].apply(lambda x: x.replace('|', ' '))

# Inisialisasi dan fit TF-IDF Vectorizer pada genre
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(movies['Genres_str'])

# Menghitung kemiripan antar film berdasarkan genre (TF-IDF)
cosine_sim_tfidf = cosine_similarity(tfidf_matrix)

# Fungsi rekomendasi film berdasarkan judul (TF-IDF)
def recommend_content_based_tfidf(title, top_n=5):
    idx = movies[movies['Title'].str.lower() == title.lower()].index
    if len(idx) == 0:
        print("Film tidak ditemukan.")
        return
    idx = idx[0]
    sim_scores = list(enumerate(cosine_sim_tfidf[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    film_idx = [i[0] for i in sim_scores]
    return movies.iloc[film_idx][['Title', 'Genres']]

# Contoh rekomendasi
judul_film = 'titanic (1997)'
print(f"Rekomendasi film mirip dengan '{judul_film}':")
display(recommend_content_based_tfidf(judul_film))